# Biopython

Als je een probleem wilt oplossen, neem dan even de tijd en kijk of iemand anders (een deel van) het probleem al voor je heeft opgelost. 
In dit deel gaan we kijken naar een library die relevant kan zijn voor je bio-informatica-uitdagingen: **Biopython**. Biopython is een hele nuttige library met functionaliteiten voor het parsen van allerlei bioinf bestanden zoals Blast outputs, PDB, FASTA, GenBank etc. Ook kan het makkelijk omgaan met portals zoals NCBI. Het heeft allerlei methodes beschikbaar voor het bewerken van sequences zoals transcriptions, translations, 
Hulpmiddelen voor het uitvoeren van algemene bewerkingen op sequenties, zoals vertaling, transcriptie en gewichtsberekeningen, code voor allignments en nog veel meer. 

zie ook https://github.com/biopython/biopython/blob/master/README.rst

Laten we eens kijken naar de Sequentie class

In [ ]:
from Bio.Seq import Seq
dir(Seq)

Zoals je ziet zit er van allerlei methodes in de Seq class. Laten we er eens een wat meer bestuderen

In [ ]:
help(Seq.ungap)

## Doe opdracht
1. Bekijk de methode `Seq.reverse_complement()` en de `methode Seq.count()`
2. Maak een test scriptje om de code uit de beschrijvingen uit te proberen
---

## Analyseer opdracht
Hieronder zie je een stukje code. Beschrijf in je eigen woorden wat de code doet en hoe het werkt

In [14]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", IUPAC.unambiguous_dna)
messenger_rna = coding_dna.transcribe()
print(repr(messenger_rna))
print(messenger_rna.translate())
print(messenger_rna.translate(to_stop=True))

Seq('AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG', IUPACUnambiguousRNA())
MAIVMGR*KGAR*
MAIVMGR


---
Biopython is ook heel handig voor het analyseren van meta data bij bijvoorbeeld een multifasta file

In [1]:
from Bio import SeqIO

for seq_record in SeqIO.parse("data/multi.fasta", "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

KY785484.1:5000-10596
Seq('GGGAGAGTGATAGGACTTTATGGCAATGGGGTCGTGATCAAAAATGGGAGTTAT...CAG', SingleLetterAlphabet())
5597
KY785481.1:5000-10058
Seq('AGTGTGGGAGAGTGATAGGACTTTATGGCAATGGGGTCGTGATCAAAAACGGGA...AGA', SingleLetterAlphabet())
5059
KY785480.1:5000-10123
Seq('CTCTGGAACAGAAATCGTCGACTTAATGTGCCATGCCACCTTCACTTCACGTCT...AGA', SingleLetterAlphabet())
5124
KY785479.1:5000-10364
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...GGC', SingleLetterAlphabet())
5365


---
of het uitlezen van genbank files


In [12]:
from Bio import SeqIO
from Bio.SeqUtils import GC

for seq_record in SeqIO.parse("data/BRCA1.gbk", "genbank"):
    print(seq_record.id)
    print(seq_record.annotations['organism'])
    print(seq_record.description)
    print(seq_record.seq[1:100])
    print(GC(seq_record.seq))
    print(seq_record.features[0])

NG_005905.2
Homo sapiens
Homo sapiens breast cancer 1, early onset (BRCA1), RefSeqGene (LRG_292) on chromosome 17
TACCTTGATTTCGTATTCTGAGAGGCTGCTGCTTAGCGGTAGCCCCTTGGTTTCCGTGGCAACGGAAAAGCGCGGGAATTACAGATAAATTAAAACTGC
42.92946088755866
type: source
location: [0:81189](+)
qualifiers:
    Key: chromosome, Value: ['17']
    Key: db_xref, Value: ['taxon:9606']
    Key: map, Value: ['17q21']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Homo sapiens']



## Visualisatie met Biopython
Biopython heeft ook een visualisatie classe `Bio.Graphics` De Bio.Graphics-module is afhankelijk van de Python-module `ReportLab`. Hoewel `ReportLab` gericht is op het produceren van PDF-bestanden, kan het ook ingekapselde PostScript- (EPS) en (SVG) -bestanden maken. Ook kan ReportLab, mits de Library (PIL) geinstalleerd is, ook  JPEG-, PNG-, GIF-, en BMP- bestanden maken


In [ ]:
from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

record = SeqIO.read("data/NC_005816.gb", "genbank")

gd_diagram = GenomeDiagram.Diagram(record.id)
gd_track_for_features = gd_diagram.new_track(1, name="Annotated Features")
gd_feature_set = gd_track_for_features.new_set()

for feature in record.features:
    if feature.type != "gene":
        #Exclude this feature
        continue
    if len(gd_feature_set) % 2 == 0:
        color = colors.blue
    else:
        color = colors.lightblue
    gd_feature_set.add_feature(feature, sigil="ARROW",
                               color=color, label=True,
                               label_size = 14, label_angle=0)

#I want to include some strandless features, so for an example
#will use EcoRI recognition sites etc.
for site, name, color in [("GAATTC","EcoRI",colors.green),
                          ("CCCGGG","SmaI",colors.orange),
                          ("AAGCTT","HindIII",colors.red),
                          ("GGATCC","BamHI",colors.purple)]:
    index = 0
    while True:
        index  = record.seq.find(site, start=index)
        if index == -1 : break
        feature = SeqFeature(FeatureLocation(index, index+len(site)))
        gd_feature_set.add_feature(feature, color=color, name=name,
                                   label=True, label_size = 10,
                                   label_color=color)
        index += len(site)

gd_diagram.draw(format="linear", pagesize='A4', fragments=4,
                start=0, end=len(record))
gd_diagram.write("plasmid_linear_nice.pdf", "PDF")
gd_diagram.write("plasmid_linear_nice.eps", "EPS")
gd_diagram.write("plasmid_linear_nice.svg", "SVG")

gd_diagram.draw(format="circular", circular=True, pagesize=(20*cm,20*cm),
                start=0, end=len(record), circle_core = 0.5)
gd_diagram.write("plasmid_circular_nice.pdf", "PDF")
gd_diagram.write("plasmid_circular_nice.eps", "EPS")
gd_diagram.write("plasmid_circular_nice.svg", "SVG")

In [10]:
%%html
<img src = 'images/plasmid_circular_nice.pdf' />

In [8]:
%%html
<img src = 'images/plasmid_linear_nice.pfd' />

Bron: https://biopython-cn.readthedocs.io/zh_CN/latest/en/chr17.html en https://youtu.be/l8wLaoEGbUI

## Lees opdracht

Bekijk http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc2 en https://biopython.org/wiki/Category%3ACookbook
en https://biopython.org/wiki/Seq
Welke code uit biopython kun je gebruiken voor de opdracht die hier beschreven staat? https://bioinf.nl/~fennaf/minor/WC02_IO2.html. En welke voor je practicum opdracht :-)
    